In [1]:
import pandas as pd
from datetime import datetime, timedelta
from prettytable import PrettyTable
import random

data = pd.read_csv("2008.csv")
carrier = pd.read_csv("AirlineCodes.csv")
airport = pd.read_csv("airports.csv")

current_month = datetime.now().month
current_day = datetime.now().day
current_time = datetime.now().time()

itinerary = data.loc[(data['Month'] == current_month) & (data['DayofMonth'] == current_day), ['UniqueCarrier', 'DepTime', 'ArrTime', 'FlightNum', 'Origin', 'Dest']]
itinerary = itinerary.dropna() 

carrier_mapping = dict(zip(carrier['Code'], carrier['Name']))

airport_mapping = dict(zip(airport['ident'].str.lstrip('K'), airport['municipality'].str.split('/ ').str[0]))

delays = data[['DepDelay']]
cancellations = data[['Cancelled']]

def get_gate():
    gate_letters = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L']
    gate_numbers = list(range(1, 60))  # Gate numbers 1 to 59

    # Randomly select a gate letter and number
    gate_letter = random.choice(gate_letters)
    gate_number = random.choice(gate_numbers)

    # Remove the selected gate letter and number from the available options
    gate_letters.remove(gate_letter)
    gate_numbers.remove(gate_number)

    return f"{gate_letter}{gate_number}"
            

def parse_time(time_str):
    try:
        return datetime.strptime(str(time_str).split('.')[0], "%H%M").time()
    except ValueError:
        pass
    
def has_delay(row): 
    for column in delays:
        if pd.notna(data.loc[row.name, column]) and data.loc[row.name, column] >= 15:
            return True
    return False

def cancelled(row):
    for column in cancellations:
        if pd.notna(data.loc[row.name, column]) and data.loc[row.name, column] != 0:
            return True
    return False

def boarding(row):
    dep_time = parse_time(row['DepTime'])
    if dep_time is not None:
        boarding_time = datetime.combine(datetime.today(), dep_time) - timedelta(minutes=45)
        return boarding_time.time() <= current_time <= dep_time
    return False

def departed(row):
    dep_time = parse_time(row['DepTime'])
    if dep_time is not None:
        push_back = datetime.combine(datetime.today(), dep_time) - timedelta(minutes=10)
        return push_back.time() <= current_time <= dep_time
    return False

def get_airline(carrier_code):
    if carrier_code in carrier_mapping:
        return carrier_mapping[carrier_code]
    return "Void"

def get_airport(id):
    if id in airport_mapping:
        return airport_mapping[id]
    return "Void"

def get_status(row):
    if has_delay(row) and not boarding(row) and not departed(row):
        return "Delayed"
    elif cancelled(row):
        return "Cancelled"
    elif boarding(row) and not departed(row):
        return "Boarding"
    elif departed(row):
        return "Departed"
    else:
        return "On Time"
        
def format_time(time): 
    return time.strftime("%I:%M %p")

def generate_arrival_board(rows):
    tableA = PrettyTable()
    tableA.field_names = ["Airline", "Flight", "Origin", "Status", "Time", "Gate"]
    
    for _, row in rows.iterrows():
        arr_time = parse_time(row['ArrTime'])
        dep_time = parse_time(row['DepTime'])
        origin = get_airport(row['Origin'])
        if arr_time is not None and dep_time is not None and origin != 'Void':
            if current_time >= dep_time and current_time <= arr_time: #arrivals
                tableA.add_row([
                    get_airline(row['UniqueCarrier']),
                    row['FlightNum'],
                    get_airport(row['Origin']),
                    get_status(row),
                    format_time(arr_time),
                    get_gate()])
    return tableA


def generate_departure_board(rows):
    tableD = PrettyTable()
    tableD.field_names = ["Airline", "Flight", "Destination", "Status", "Time", "Gate"]

    for _, row in rows.iterrows():
        dep_time = parse_time(row['DepTime'])
        destination = get_airport(row['Dest'])
        if dep_time is not None and destination != 'Void':
            if current_time <= dep_time: 
                tableD.add_row([
                    get_airline(row['UniqueCarrier']),
                    row['FlightNum'],
                    get_airport(row['Dest']),
                    get_status(row),
                    format_time(dep_time),
                    get_gate()])
    return tableD

def arrivals(airport):
    filtered_rows = itinerary[itinerary['Dest'] == airport]
    return generate_arrival_board(filtered_rows)

def departures(airport):
    filtered_rows = itinerary[itinerary['Origin'] == airport]
    return generate_departure_board(filtered_rows)

arrival_board = arrivals('SLC')
departure_board = departures('SLC')
display_time = format_time(current_time)

print(display_time)

print("\nArrivals:")
print(arrival_board)

print("\nDepartures:")
print(departure_board)


07:53 PM

Arrivals:
+-------------------+--------+-------------------+---------+----------+------+
|      Airline      | Flight |       Origin      |  Status |   Time   | Gate |
+-------------------+--------+-------------------+---------+----------+------+
|     Southwest     |  2710  |     Las Vegas     | On Time | 09:37 PM | F13  |
|     Southwest     |  1785  |    Los Angeles    | Delayed | 09:40 PM |  I4  |
|     Southwest     |  266   |      Chicago      | On Time | 09:29 PM | D22  |
|     Southwest     |  3079  |      Portland     | Delayed | 08:51 PM | C48  |
|     Southwest     |  3494  |      Phoenix      | On Time | 08:06 PM | K25  |
|     Southwest     |  2381  |      Seattle      | On Time | 10:25 PM |  K2  |
|     ExpressJet    |  7672  |     Las Vegas     | On Time | 09:30 PM | G34  |
|     ExpressJet    |  2271  |      Houston      | Delayed | 08:56 PM | L14  |
|      SkyWest      |  3606  |     St George     | On Time | 08:01 PM |  L8  |
|      SkyWest      |  3614  |  